# Choosing GPU device

In [ ]:
import os
import torch

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="1"  
#
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="cuda:1"  
#
#
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
#os.environ["device"]="cuda:1"  # device = "cuda:1"

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=6

#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=cuda:1

#torch.device("cuda:1")

print(torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    info = torch.cuda.get_device_properties(i)
    print(f"CUDA:{i} {info.name}, {info.total_memory / 1024 ** 2}MB")

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=6
1
CUDA:0 NVIDIA A40, 45434.1875MB


/u/home/wyo/.conda/envs/mesh_gnn_organ/lib/python3.8/site-packages/torch/cuda/__init__.py:125: UserWarning: 
NVIDIA A40 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA A40 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [ ]:
# GPU picking
# http://stackoverflow.com/a/41638727/419116
# Nvidia-smi GPU memory parsing.
# Tested on nvidia-smi 370.23

def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")

def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

def gpu_memory_map():
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    gpu_output = output[output.find("GPU Memory"):]
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    rows = gpu_output.split("\n")
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

def pick_gpu_lowest_memory():
    """Returns GPU with the least allocated memory"""

    memory_gpu_map = [(memory, gpu_id) for (gpu_id, memory) in gpu_memory_map().items()]
    best_memory, best_gpu = sorted(memory_gpu_map)[0]
    return best_gpu

def setup_one_gpu():
    assert not 'tensorflow' in sys.modules, "GPU setup must happen before importing TensorFlow"
    gpu_id = pick_gpu_lowest_memory()
    print("Picking GPU "+str(gpu_id))
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

def setup_no_gpu():
    if 'tensorflow' in sys.modules:
        print("Warning, GPU setup must happen before importing TensorFlow")
    os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [ ]:
setup_one_gpu()

# Choosing Conda Environment

In [3]:
os.environ["CONDA_EXE"] = "/u/home/wyo/.conda/envs/mesh_gnn_organ/bin/conda"
os.environ["CONDA_PREFIX"] = "/u/home/wyo/.conda/envs/mesh_gnn_organ"
!conda info --envs

# conda environments:
#
base                     /opt/anaconda3
mesh_gnn_organ        *  /u/home/wyo/.conda/envs/mesh_gnn_organ
model_gnn_organ          /u/home/wyo/.conda/envs/model_gnn_organ
organ_mesh_gnn_model     /u/home/wyo/.conda/envs/organ_mesh_gnn_model



# Imports

In [4]:
import subprocess, re, os, sys
import matplotlib.pyplot as plt
import nibabel as nib
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pyvista
from skimage.measure import marching_cubes
import numpy as np
import shutil
from itertools import permutations
import json
import torch 
import pandas as pd
import copy
import os
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [5]:
import open3d as o3d

# Functions

In [9]:
root_path = '/data0/practical-wise2223/organ_mesh/'

#Reading and Loading
def pad_edge_list(edges):
    padding = np.ones(edges.shape[0], int)*3
    edges_w_padding = np.vstack((padding, edges.T)).T
    return edges_w_padding

def organs_data(abdominal_segmentations_data, dir):
    try:
        verts1, faces1, norms1, vals1 = marching_cubes(abdominal_segmentations_data==1, level=0, step_size=1)
        verts2, faces2, norms2, vals2 = marching_cubes(abdominal_segmentations_data==2, level=0, step_size=1)
        verts3, faces3, norms3, vals3 = marching_cubes(abdominal_segmentations_data==3, level=0, step_size=1)
        verts4, faces4, norms4, vals4 = marching_cubes(abdominal_segmentations_data==4, level=0, step_size=1)
        verts5, faces5, norms5, vals5 = marching_cubes(abdominal_segmentations_data==5, level=0, step_size=1)
        
        verts1 = verts1/np.array(abdominal_segmentations_data.shape) 
        verts2 = verts2/np.array(abdominal_segmentations_data.shape) 
        verts3 = verts3/np.array(abdominal_segmentations_data.shape) 
        verts4 = verts4/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
        verts5 = verts5/np.array(abdominal_segmentations_data.shape) # to normalize ponit coordinate in [0,1]
        
        edges1 = np.concatenate((faces1[:,:2], faces1[:,1:]), axis=0)
        edges2 = np.concatenate((faces2[:,:2], faces2[:,1:]), axis=0)
        edges3 = np.concatenate((faces3[:,:2], faces3[:,1:]), axis=0)
        edges4 = np.concatenate((faces4[:,:2], faces4[:,1:]), axis=0)
        edges5 = np.concatenate((faces5[:,:2], faces5[:,1:]), axis=0)

        lines1 = np.concatenate((np.int32(2*np.ones((edges1.shape[0],1))), edges1), 1)
        lines2 = np.concatenate((np.int32(2*np.ones((edges2.shape[0],1))), edges2), 1)
        lines3 = np.concatenate((np.int32(2*np.ones((edges3.shape[0],1))), edges3), 1)
        lines4 = np.concatenate((np.int32(2*np.ones((edges4.shape[0],1))), edges4), 1)
        lines5 = np.concatenate((np.int32(2*np.ones((edges5.shape[0],1))), edges5), 1)
        
        mesh1 = pyvista.PolyData(verts1, pad_edge_list(faces1))
        mesh2 = pyvista.PolyData(verts2, pad_edge_list(faces2))
        mesh3 = pyvista.PolyData(verts3, pad_edge_list(faces3))
        mesh4 = pyvista.PolyData(verts4, pad_edge_list(faces4))
        mesh5 = pyvista.PolyData(verts5, pad_edge_list(faces5))

        mesh1.lines = lines1.flatten()
        mesh2.lines = lines2.flatten()
        mesh3.lines = lines3.flatten()
        mesh4.lines = lines4.flatten()
        mesh5.lines = lines5.flatten()

        # verts = [verts1, verts2, verts3, verts4, verts5]
        # edges = [edges1, edges2, edges3, edges4, edges5]
        # faces = [faces1, faces2, faces3, faces4, faces5]
        # lines = [lines1, lines2, lines3, lines4, lines5]
        # meshes = [mesh1, mesh2, mesh3, mesh4, mesh5]
        # norms = [norms1, norms2, norms3, norms4, norms5]
        # vals = [vals1, vals2, vals3, vals4, vals5]

        #verts = [verts1]
        #edges = [edges1]
        #faces = [faces1]
        #lines = [lines1]
        #meshes = [mesh1]
        #norms = [norms1]
        #vals = [vals1]
        
        organ1 = [verts1, edges1, faces1, lines1, mesh1, norms1, vals1]
        organ2 = [verts2, edges2, faces2, lines2, mesh2, norms2, vals2]
        organ3 = [verts3, edges3, faces3, lines3, mesh3, norms3, vals3]
        organ4 = [verts4, edges4, faces4, lines4, mesh4, norms4, vals4]
        organ5 = [verts5, edges5, faces5, lines5, mesh5, norms5, vals5]

        #data = [verts, edges, faces, lines, meshes, norms, vals]
        organs_data = [organ1, organ2, organ3, organ4, organ5]
        return organs_data
    except:
        x=0


def load_point_cloud(i, path):
    paint = np.asarray([i*130, i*100, i*120])/255.0
    pcd = o3d.io.read_point_cloud(path)
    pcd.estimate_normals()
    pcd_normals = pcd.paint_uniform_color(paint)
    return pcd_normals  

def load_point_clouds(voxel_size, organ_index):
    paint1 = np.asarray([0, 255, 0])/255.0
    paint2 = np.asarray([0, 0, 0])/255.0
    paint3 = np.asarray([0, 0, 255])/255.0
    paint4 = np.asarray([255, 0, 0])/255.0
    paint5 = np.asarray([255, 0, 255])/255.0
    paint6 = np.asarray([255, 255, 0])/255.0
    paint7 = np.asarray([0, 255, 255])/255.0
    paints = [paint1, paint2, paint3, paint4, paint5, paint6, paint7]
    pcd = o3d.geometry.PointCloud()
    pcds = []
    i = 0
    decimations_path = "organ_decimations_ply"
    registrations_path = "organ_registrations_ply"
    gendered_registrations_path = "gendered_organ_registrations_ply"
    organs = ["liver_mesh.ply", "spleen_mesh.ply", "left_kidney_mesh.ply", "right_kidney_mesh.ply", "pancreas_mesh.ply"]
    path = registrations_path
    #path = "server_regs"
    #path = "organ_registrations_ply_p2p"
    for _root, dirs, _file_list in os.walk(path):
        for dir in dirs:
            if(dir != ".ipynb_checkpoints"):
                paint = paints[i]
                i = i + 1
                pcd = o3d.io.read_point_cloud(os.path.join(root_path, path, dir, organs[organ_index]))
                #pcd.estimate_normals()
                #pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
                pcd_down = pcd.paint_uniform_color(paint)
                pcds.append(pcd_down)
    return pcds

#Processing and Analysis
def triangle_mesh(data):
    verts = data[0]
    faces = data[2]
    mesh_list = []
    mesh_list.append(o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts)), triangles = o3d.utility.Vector3iVector(np.asarray(faces))))
    return mesh_list

def get_min_point_triangle():
    min_vertices = [np.nan, np.nan, np.nan, np.nan, np.nan]
    min_triangles = [np.nan, np.nan, np.nan, np.nan, np.nan]
    sum_vertices = [0,0,0,0,0]
    sum_triangles = [0,0,0,0,0]
    count = [0,0,0,0,0]
    min_vertices_mesh = [np.nan, np.nan, np.nan, np.nan, np.nan]
    min_triangles_mesh = [np.nan, np.nan, np.nan, np.nan, np.nan]
    empty_files = []
    bad_qual = []
    path_to_segmentations = "organ_segmentations"
    for _root, dirs, _file_list in os.walk(path_to_segmentations):
        for dir in dirs:
            if(os.path.exists(os.path.join(root_path, path_to_segmentations, dir, "prd.nii.gz"))):
                abdominal_segmentations = nib.load(os.path.join(path_to_segmentations, dir, "prd.nii.gz"))
                abdominal_segmentations_data = abdominal_segmentations.get_fdata()

                data = organs_data(abdominal_segmentations_data, dir)
                if(data is not None):
                    mesh_list = triangle_mesh(data[0], data[2])
                    
                    for i in range(0, len(mesh_list)):
                        sum_vertices[i] = sum_vertices[i] + len(np.asarray(mesh_list[i].vertices)) 
                        sum_triangles[i] = sum_triangles[i] + len(np.asarray(mesh_list[i].triangles))
                        count[i] = count[i] + 1 
                        if(len(np.asarray(mesh_list[i].vertices)) < 2000):
                            print("Less than 2000 points: {}".format(dir))
                            bad_qual.append(dir)
                        else:
                            if(np.isnan(min_triangles[i])):
                                min_triangles[i] = len(np.asarray(mesh_list[i].triangles))
                                min_triangles_mesh[i] = dir
                            if(len(np.asarray(mesh_list[i].triangles)) < min_triangles[i]):
                                min_triangles[i] = len(np.asarray(mesh_list[i].triangles))
                                min_triangles_mesh[i] = dir

                            if(np.isnan(min_vertices[i])):
                                min_vertices[i] = len(np.asarray(mesh_list[i].vertices))
                                min_vertices_mesh[i] = dir
                            if(len(np.asarray(mesh_list[i].vertices)) < min_vertices[i]):
                                min_vertices[i] = len(np.asarray(mesh_list[i].vertices))
                                min_vertices_mesh[i] = dir
            else:
                empty_files.append(dir)

    minimums = [min_vertices, min_triangles]
    vert_avg = [i / j for i, j in zip(sum_vertices, count)]
    triangle_avg = [[i / j for i, j in zip(sum_triangles, count)]]
    averages = [vert_avg, triangle_avg]
    files = [min_vertices_mesh, min_triangles_mesh]
    return files, minimums, averages, empty_files, bad_qual

def get_average(lst):
    return sum(lst) / len(lst)

def baseline_L1loss(data, label):
    mean = get_average(data[label])
    total = 0
    for label in data[label]:
        total = total + abs(label - mean)
    return total/len(data[label])

def normalize(data, column):
    data_without_nans = data[data.notna()]
    data_without_nans[column] = MinMaxScaler().fit_transform(np.array(data_without_nans[column]).reshape(-1,1))
    data_without_nans.to_csv(os.path.join(root_path, 'normalized_basic_features.csv')) 
    

#Decimation
def dec_triangle_mesh_ply(dec):
    segmentations_path = "organ_segmentations"
    decimations_path = "organ_decimations_ply"
    organs = ["liver_mesh.ply", "spleen_mesh.ply", "left_kidney_mesh.ply", "right_kidney_mesh.ply", "pancreas_mesh.ply"]
    empty_files = []
    path = root_path
    if(not os.path.exists(os.path.join(path, decimations_path))):
                                os.mkdir(os.path.join(path, decimations_path))
    dirs = next(os.walk(os.path.join(path, segmentations_path)))[1]
    for dir in dirs:
        dec_already_done = True
        for i in range(0, len(organs)):
                if(not os.path.exists(os.path.join(path, decimations_path, str(dir) , organs[i]))):
                    dec_already_done = False
        
        if(not dec_already_done):
            if(os.path.exists(os.path.join(path, segmentations_path, dir, "prd.nii.gz"))):
                abdominal_segmentations = nib.load(os.path.join(path, segmentations_path, dir, "prd.nii.gz"))
                abdominal_segmentations_data = abdominal_segmentations.get_fdata()
                try:
                    organs_info = organs_data(abdominal_segmentations_data, dir)
                    if(organs_info is not None):
                        for i in range(0, len(organs_info)):
                            data = organs_info[i]
                            if(data is not None):
                                mesh_list = triangle_mesh(data)
                                for j in range(0, len(mesh_list)):
                                    dec_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh_list[j], dec)
                                    dec_mesh_path = os.path.join(path, decimations_path, dir, organs[i])
                                    if(not os.path.exists(os.path.join(path, decimations_path, str(dir)))):
                                        os.mkdir(os.path.join(path, decimations_path, str(dir)))
                                    o3d.io.write_triangle_mesh(dec_mesh_path, dec_mesh, print_progress=True, write_ascii=True)
                            if(data is None):
                                empty_files.append(dir)
                    else:
                        empty_files.append(dir)
                except:
                    x = str(dir)
                    print("Error!!!: %s" %x)
                    empty_files.append(dir)
    return empty_files

def gender_split_decimations():
    path = root_path
    decimations_path = "organ_decimations_ply/"
    
 

    male_mesh_ids = np.loadtxt(os.path.join(path, "male_mesh_ids.csv"),
                 delimiter=",", dtype=str)
    female_mesh_ids = np.loadtxt(os.path.join(path, "female_mesh_ids.csv"),
                 delimiter=",", dtype=str)
    dirs = next(os.walk(os.path.join(path, decimations_path)))[1]
    if(not os.path.exists(os.path.join(path, "male_organ_decimations_ply/"))):
        os.mkdir(os.path.join(path, "male_organ_decimations_ply/"))
    if(not os.path.exists(os.path.join(path, "female_organ_decimations_ply/"))):
        os.mkdir(os.path.join(path, "female_organ_decimations_ply/"))
    
    for dir in dirs:
        if(dir in male_mesh_ids):
            if(not os.path.exists(os.path.join(path, "male_organ_decimations_ply/", str(dir)))):
                os.mkdir(os.path.join(path, "male_organ_decimations_ply/", str(dir)))
            files = next(os.walk(os.path.join(path, decimations_path, str(dir))))[2]
            for file in files:
                src = os.path.join(path, decimations_path, str(dir), str(file))
                dst = os.path.join(path, "male_organ_decimations_ply/", str(dir), str(file))
                if(not os.path.exists(os.path.join(path, "male_organ_decimations_ply/", str(dir), str(file)))):
                    shutil.copyfile(src, dst)
        if(dir in female_mesh_ids):
            if(not os.path.exists(os.path.join(path, "female_organ_decimations_ply/", str(dir)))):
                os.mkdir(os.path.join(path, "female_organ_decimations_ply/", str(dir)))
            files = next(os.walk(os.path.join(path, decimations_path, str(dir))))[2]
            for file in files:
                src = os.path.join(path, decimations_path, str(dir), str(file))
                dst = os.path.join(path, "female_organ_decimations_ply/", str(dir), str(file))
                if(not os.path.exists(os.path.join(path, "female_organ_decimations_ply/", str(dir), str(file)))):
                    shutil.copyfile(src, dst)
    
    male_dirs = len(next(os.walk(os.path.join(path, "male_organ_decimations_ply/")))[1])
    female_dirs = len(next(os.walk(os.path.join(path, "female_organ_decimations_ply/")))[1])
    male_count = len(male_mesh_ids)
    female_count = len(female_mesh_ids)
    stats = [male_count, male_dirs, female_count, female_dirs]
    print("done")
    return stats
  

#Registeration  
def pairwise_registration(source, target, voxel_size, max_correspondence_distance_coarse, max_correspondence_distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, voxel_size, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id], voxel_size, max_correspondence_distance_coarse, max_correspondence_distance_fine)
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph



def register_meshes(gender):
    path = root_path
    decimations_path = f'{gender}_organ_decimations_ply' #change for ungendered
    registrations_path = "gendered_organ_registrations_ply" #change for ungendered
    organs = ["liver_mesh.ply", "spleen_mesh.ply", "left_kidney_mesh.ply", "right_kidney_mesh.ply", "pancreas_mesh.ply"]
    targets = []
    #sources_names = []
    empty_files = 0
    
    if(not os.path.exists(os.path.join(path, registrations_path))):
        os.mkdir(os.path.join(path, registrations_path))
        
            
    pcd = o3d.geometry.PointCloud()
    dirs = next(os.walk(os.path.join(path, decimations_path)))[1]
    #target_index = next(os.walk(os.path.join(path, decimations_path)))[1][0]
    if(gender == "female"):
        target_index = "2635627"
    if(gender == "male"):
        target_index = "2901448"
    print(target_index)
    
    for i in range(0, len(organs)):
        target_path = os.path.join(path, decimations_path, target_index, organs[i])
        target = load_point_cloud(0, target_path)
        if(not os.path.exists(os.path.join(path, registrations_path, target_index))):
            os.mkdir(os.path.join(path, registrations_path, target_index))
        o3d.io.write_point_cloud(os.path.join(path, registrations_path, target_index , organs[i]), target, print_progress=True, write_ascii=True)
        targets.append(target)
        #sources_names.append(organs[i])
         
    dirs.remove(target_index)   
    colour_index = 1
    voxel_size = 0.02
    max_correspondence_distance_coarse = voxel_size * 15
    max_correspondence_distance_fine = voxel_size * 1.5
    
    for dir in dirs:
        reg_already_done = True
        for i in range(0, len(organs)):
            if(not os.path.exists(os.path.join(path, registrations_path, str(dir) , organs[i]))):
                reg_already_done = False
                
        if(not reg_already_done):
            for i in range(0, len(organs)):
                if(os.path.exists(os.path.join(path, decimations_path, str(dir) , organs[i]))):
                    pcds = []
                    source_path = os.path.join(path, decimations_path, str(dir) , organs[i])
                    source = load_point_cloud(colour_index, source_path)
                    colour_index = colour_index + 1
                    pcds = [targets[i], source]
                    #draw_preproccessing(pcds[1], pcds[0])
                    
                    #pairwise reg
                    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
                        pose_graph = full_registration(pcds, voxel_size, max_correspondence_distance_coarse, max_correspondence_distance_fine)
                    option = o3d.pipelines.registration.GlobalOptimizationOption(max_correspondence_distance=max_correspondence_distance_fine,
                                                                                 edge_prune_threshold=0.05, reference_node=0)
                    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
                        o3d.pipelines.registration.global_optimization(pose_graph,
                            o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
                            o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
                            option)
                    
                    #transformation_icp, information_icp = pairwise_registration(pcds[0], pcds[1])
                        
                    #for point_id in range(len(pcds)):
                    pcds[1].transform(pose_graph.nodes[1].pose)
                    #print(pose_graph.nodes[1].pose)
                    #draw_result(pcds[1], pcds[0])
                    #source_temp = copy.deepcopy(pcds[1])
                    #target_temp = copy.deepcopy(pcds[0])
                    #source_temp.paint_uniform_color([255, 0, 0])
                    #target_temp.paint_uniform_color([0, 255, 0])
                    #o3d.visualization.draw_geometries([source_temp, target_temp])
                    
                    if(not os.path.exists(os.path.join(path, registrations_path, str(dir)))):
                                os.mkdir(os.path.join(path, registrations_path, str(dir)))
                    o3d.io.write_point_cloud(os.path.join(path, registrations_path, str(dir) , organs[i]), pcds[1], print_progress=True, write_ascii=True)

#Visualization
def draw_preproccessing(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([255, 0, 0])
    target_temp.paint_uniform_color([0, 255, 0])
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def draw_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([255, 0, 0])
    target_temp.paint_uniform_color([0, 255, 0])
    o3d.visualization.draw_geometries([source_temp, target_temp])
                    
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([255, 0, 0])
    target_temp.paint_uniform_color([0, 255, 0])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

#Progress checking
def finished():
    path = root_path
    segmentations_path = "organ_segmentations"
    decimations_path = "organ_decimations_ply"
    registrations_path = "gendered_organ_registrations_ply"
    chossen_path = registrations_path
    organs = ["liver_mesh.ply", "spleen_mesh.ply", "left_kidney_mesh.ply", "right_kidney_mesh.ply", "pancreas_mesh.ply"]
    finished_status = True
    finished = 0
    unfinished = 0
    unfinished_ids = {}
    unfinished_dirs = []

    dirs = next(os.walk(os.path.join(path, chossen_path)))[1]
    for dir in dirs:
        finished_status = True
        unfinished_organs = []
        for i in range(0, len(organs)):
            if(not os.path.exists(os.path.join(path, chossen_path, str(dir) , organs[i]))):
                finished_status = False
                unfinished_organs.append(organs[i])
        if(finished_status):
            finished = finished + 1
        else:
            unfinished = unfinished + 1
            unfinished_organs.append(len(unfinished_organs))
            unfinished_ids[dir] = [unfinished_organs]
            unfinished_dirs.append(dir)

    return finished, unfinished, unfinished_ids, unfinished_dirs

# Progress Checker

In [7]:
x, y, z, q= finished()
print(x, y)

30379 0


# Visualizations

In [5]:
# with open3d: point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(verts1))

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [6]:
# with open3d: graph
lines = o3d.utility.Vector2iVector(np.asarray(edges1))
line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(verts1)),
        lines=lines)

line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines1))])

o3d.visualization.draw_geometries([line_set])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [15]:
# with open3d: triangle mesh
mesh = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(np.asarray(verts2)), triangles = o3d.utility.Vector3iVector(np.asarray(faces2)))
#mesh.compute_vertex_normals()
#mesh.paint_uniform_color([0, 1, 0]) 
#vis = o3d.visualization.Visualizer()
#vis.create_window()
#vis.add_geometry(mesh)
# vis.update_geometry()
#vis.poll_events()
#vis.update_renderer()
#path = "liver_mesh.jpg"
#vis.capture_screen_image(path)
#vis.destroy_window()
mesh
# o3d.visualization.draw_geometries([mesh])

TriangleMesh with 7904 points and 15804 triangles.

# Data Processing

In [ ]:
data = pd.read_csv("/data0/practical-wise2223/organ_mesh/basic_features.csv")
normalized_data = pd.read_csv("/data0/practical-wise2223/organ_mesh/normalized_basic_features.csv")
normalize(data, "21003-2.0")
raw_loss  = baseline_L1loss(data, "21003-2.0")
normalized_loss  = baseline_L1loss(normalized_data, "21003-2.0")

# Decimation

In [ ]:
minimums

[[2488, 6, 6, 22, 38], [4974, 8, 8, 40, 72]]

In [6]:
empty_files = dec_triangle_mesh_ply(2000)

# Registeration

In [10]:
stats = gender_split_decimations()
stats

done


[14653, 14653, 15729, 15726]

In [13]:
register_meshes("male")

2901448


In [8]:
register_meshes("female")

2635627


In [8]:
path = "/data0/practical-wise2223/organ_mesh/organ_registrations_ply/"
print(len(next(os.walk(path))[1]))

30721


In [1]:
path = "/data0/practical-wise2223/organ_mesh/gendered_organ_registrations_ply/"
print(len(next(os.walk(path))[1]))

30379
